In [2]:
# importamos librerias requeridas para el ETL
import numpy as np
import pandas as pd
import json
import re
import ast
from pandas import json_normalize

In [3]:
# Cargamos los datos
df = pd.read_csv(r'C:\Users\gasty\Desktop\Data\Dataset\movies_dataset.csv')
df_credits = pd.read_csv(r'C:\Users\gasty\Desktop\Data\Dataset\credits.csv')

C:\Users\gasty\AppData\Local\Temp\ipykernel_14976\3832524643.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'C:\Users\gasty\Desktop\Data\Dataset\movies_dataset.csv')


# Analisis general de la informacion

In [4]:
print(df.shape)
df.head()

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

In [6]:
df.describe()

,revenue,runtime,vote_average,vote_count
count,4.546000e+04,45203.000000,45460.000000,45460.000000
mean,1.120935e+07,94.128199,5.618207,109.897338
std,6.433225e+07,38.407810,1.924216,491.310374
min,0.000000e+00,0.000000,0.000000,0.000000
25%,0.000000e+00,85.000000,5.000000,3.000000
50%,0.000000e+00,95.000000,6.000000,10.000000
75%,0.000000e+00,107.000000,6.800000,34.000000
max,2.787965e+09,1256.000000,10.000000,14075.000000


In [7]:
tipo_texto = df.select_dtypes(include=["object"]).columns
data_texto = df[tipo_texto]
data_texto.describe()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,spoken_languages,status,tagline,title,video
count,45466,4494,45466,45466,7782,45466,45449,45455,45466,44512,45461,45080,45463,45463,45379,45460,45379,20412,45460,45460
unique,5,1698,1226,4069,7673,45436,45417,92,43373,44307,44176,45024,22708,2393,17336,1931,6,20283,42277,2
top,False,"{'id': 415931, 'name': 'The Bowery Boys', 'pos...",0,"[{'id': 18, 'name': 'Drama'}]",http://www.georgecarlin.com,141971,tt1180333,en,Hamlet,No overview found.,0.0,/5D7UBSEgdyONE6Lql6xS7s6OLcW.jpg,[],"[{'iso_3166_1': 'US', 'name': 'United States o...",2008-01-01,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Based on a true story.,Cinderella,False
freq,45454,29,36573,5000,12,3,3,32269,8,133,34,5,11875,17851,136,22395,45014,7,11,45367


In [13]:
df.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [9]:
print(df_credits.shape)
df_credits.head()

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [10]:
df_credits.describe() 

,id
count,45476.000000
mean,108345.997537
std,112443.796536
min,2.000000
25%,26443.250000
50%,60002.500000
75%,157302.000000
max,469172.000000


In [11]:
credit_texto = df_credits.select_dtypes(include=['object']).columns 
data_text_credit = df_credits[credit_texto]
data_text_credit.describe()

,cast,crew
count,45476,45476
unique,43019,44669
top,[],[]
freq,2418,771


In [12]:
df_credits.isnull().sum()

cast    0
crew    0
id      0
dtype: int64

# Transformacion de movies

1- Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [6]:
df = df.drop(['adult', 'imdb_id', 'video', 'original_title', 'homepage', 'poster_path'], axis=1)

2- Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [7]:
# combercion a valores numericos
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

In [8]:
# Rellenar los datos nulos con 0 solo en las columnas seleccionadas
columnas_rellenar = ['revenue', 'budget']
df[columnas_rellenar] = df[columnas_rellenar].fillna(0)

3- Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [9]:
df['return'] = np.where((df['revenue'] == 0) | (df['budget'] == 0), 0, df['revenue'] / df['budget'])

4- Los valores nulos del campo release date deben eliminarse.
las fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [10]:
# Elimina los campos con valores nulos en la columna "release_date"
df = df.dropna(subset=['release_date'])

In [11]:
# Busca valores dentro de dicha columna que no se puedan transformar a formato de fecha y imprime sus id con estos valores
error_rows = df[~df['release_date'].fillna('').astype(str).str.match(r'^\d{4}-\d{2}-\d{2}$')]
print(error_rows['release_date'])


19730     1
29503    12
35587    22
Name: release_date, dtype: object


In [12]:
# Elimina los campos que contienen estos valores
df = df[~df['release_date'].isin(['1', '12', '22'])]

In [13]:
# Cambia el formato de los valores de esta columna a fechas
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

In [14]:
# Comprobacion de formatos de estos valores
df['release_date'].dtype

dtype('<M8[ns]')

In [15]:
# Crea una nueva columna extrayendo solo el año de "release_date"
df['release_year'] = df['release_date'].dt.year

5- Desanidar
Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [16]:
# Convierte una cadena de texto que representa un diccionario en un objeto de diccionario en Python.
# Si el valor es nulo, devuelve None.
# Si el valor no es nulo, utiliza la evaluación literal de la biblioteca ast para convertir la cadena en un diccionario válido en Python.
def parse_dict_string(value):
    if pd.isna(value):
        return None
    else:
        return ast.literal_eval(value)

In [17]:
# Aplica la funcion a dichas columnas
columnas_desa = ['belongs_to_collection', 'genres', 'production_companies', 'production_countries','spoken_languages']
for columna in columnas_desa:
    df[columna] = df[columna].apply(parse_dict_string)

In [19]:
# Normaliza las columnas del DataFrame en formato JSON y rellena los valores nulos con una cadena vacía.
# Utiliza pd.json_normalize para convertir los datos JSON en columnas separadas.
df_belongs_to_collection = pd.json_normalize(df['belongs_to_collection']).fillna('')
df_genres = pd.json_normalize(df['genres']).fillna('')
df_production_companies = pd.json_normalize(df['production_companies']).fillna('')
df_production_countries = pd.json_normalize(df['production_countries']).fillna('')
df_spoken_languages = pd.json_normalize(df['spoken_languages']).fillna('')

In [20]:

# Función que une los valores de un diccionario en una sola cadena separada por comas
# Recibe una fila, una clave 'key' y una clave opcional 'key_aux'
# Itera sobre el diccionario y obtiene los valores correspondientes a 'key', 'name' y 'key_aux' (si se proporciona)
# Devuelve tres cadenas separadas por comas que contienen los valores obtenidos
def unir_valores(fila, key, key_aux=None):
    ids = []
    names = []
    aux = []

    for dic in fila:
        if isinstance(dic, dict):
            ids.append(str(dic.get(key, '')))
            names.append(dic.get('name', ''))
            if key_aux is not None and key_aux in dic:
                aux.append(str(dic.get(key_aux, '')))         
    return ','.join(ids), ','.join(names), ','.join(aux)

Belongs_to_collection

In [21]:
df_belongs_to_collection

,id,name,poster_path,backdrop_path
0,10194.0,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,,,,
2,119050.0,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,,,,
4,96871.0,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
...,...,...,...,...
45371,,,,
45372,,,,
45373,,,,
45374,,,,


Genres

In [22]:
df_genres

,0,1,2,3,4,5,6,7
0,"{'id': 16, 'name': 'Animation'}","{'id': 35, 'name': 'Comedy'}","{'id': 10751, 'name': 'Family'}",,,,,
1,"{'id': 12, 'name': 'Adventure'}","{'id': 14, 'name': 'Fantasy'}","{'id': 10751, 'name': 'Family'}",,,,,
2,"{'id': 10749, 'name': 'Romance'}","{'id': 35, 'name': 'Comedy'}",,,,,,
3,"{'id': 35, 'name': 'Comedy'}","{'id': 18, 'name': 'Drama'}","{'id': 10749, 'name': 'Romance'}",,,,,
4,"{'id': 35, 'name': 'Comedy'}",,,,,,,
...,...,...,...,...,...,...,...,...
45371,"{'id': 18, 'name': 'Drama'}","{'id': 28, 'name': 'Action'}","{'id': 10749, 'name': 'Romance'}",,,,,
45372,"{'id': 18, 'name': 'Drama'}",,,,,,,
45373,"{'id': 28, 'name': 'Action'}","{'id': 18, 'name': 'Drama'}","{'id': 53, 'name': 'Thriller'}",,,,,
45374,,,,,,,,


In [23]:
# Aplicar la función "unir_valores" a cada fila del DataFrame y crear nuevas columnas "id" y "name", luego extrae solo las columnas "id" y "name" a un nuevo DF
df_genres[['Id', 'name', 'aux']] = df_genres.apply(lambda row: unir_valores(row, 'id'), axis=1, result_type='expand')
DF_Genres = df_genres[['Id', 'name']]

In [24]:
DF_Genres

,Id,name
0,"16,35,10751","Animation,Comedy,Family"
1,"12,14,10751","Adventure,Fantasy,Family"
2,"10749,35","Romance,Comedy"
3,"35,18,10749","Comedy,Drama,Romance"
4,35,Comedy
...,...,...
45371,"18,28,10749","Drama,Action,Romance"
45372,18,Drama
45373,"28,18,53","Action,Drama,Thriller"
45374,,


Production_companies

In [25]:
# Aplicar la función "unir_valores" a cada fila del DataFrame y crear nuevas columnas "id" y "name", luego extrae solo las columnas "id" y "name" a un nuevo DF
df_production_companies[['Id', 'name', 'aux']] = df_production_companies.apply(lambda row: unir_valores(row, 'id'), axis=1, result_type='expand')
DF_Production_Companies = df_production_companies[['Id', 'name']]

In [26]:
DF_Production_Companies

,Id,name
0,3,Pixar Animation Studios
1,"559,2550,10201","TriStar Pictures,Teitler Film,Interscope Commu..."
2,"6194,19464","Warner Bros.,Lancaster Gate"
3,306,Twentieth Century Fox Film Corporation
4,"5842,9195","Sandollar Productions,Touchstone Pictures"
...,...,...
45371,"7025,10163,16323,38978","Westdeutscher Rundfunk (WDR),Working Title Fil..."
45372,19653,Sine Olivia
45373,6165,American World Pictures
45374,88753,Yermoliev


Production_countries

In [27]:
# Aplicar la función "unir_valores" a cada fila del DataFrame y crear nuevas columnas "id" y "name", luego extrae solo las columnas "id" y "name" a un nuevo DF
df_production_countries[['Id', 'name', 'aux']] = df_production_countries.apply(lambda row: unir_valores(row, 'iso_3166_1'), axis=1, result_type='expand')
DF_Production_Countries = df_production_countries[['Id', 'name']]

In [28]:
DF_Production_Countries

,Id,name
0,US,United States of America
1,US,United States of America
2,US,United States of America
3,US,United States of America
4,US,United States of America
...,...,...
45371,"CA,DE,GB,US","Canada,Germany,United Kingdom,United States of..."
45372,PH,Philippines
45373,US,United States of America
45374,RU,Russia


Spoken_languages

In [29]:
# Aplicar la función "unir_valores" a cada fila del DataFrame y crear nuevas columnas "id" y "name", luego extrae solo las columnas "id" y "name" a un nuevo DF
df_spoken_languages[['Id', 'name', 'aux']] = df_spoken_languages.apply(lambda row: unir_valores(row, 'iso_639_1'), axis=1, result_type='expand')
DF_Spoken_Languages = df_spoken_languages[['Id', 'name']]

In [30]:
DF_Spoken_Languages

,Id,name
0,en,English
1,"en,fr","English,Français"
2,en,English
3,en,English
4,en,English
...,...,...
45371,en,English
45372,tl,
45373,en,English
45374,,


# Transformacion en credits

In [31]:
# Función que extrae los directores de una fila
# Recibe una fila que debe contener una lista de diccionarios
# Itera sobre los diccionarios en la fila y verifica si el valor de la clave 'job' es 'Director'
# Si cumple la condición, agrega el valor de la clave 'name' a una lista de directores
# Devuelve una cadena que contiene los nombres de los directores separados por comas
def extraer_directores(fila):
    directores = []
    for dic in fila:
        if isinstance(dic, dict) and dic.get('job') == 'Director':
            directores.append(dic.get('name', ''))
    return ', '.join(directores)

In [32]:
# Aplica la funcion parse_dict_string
# Normaliza las columnas del DataFrame en formato JSON y rellena los valores nulos con una cadena vacía.
# Utiliza pd.json_normalize para convertir los datos JSON en columnas separadas.
df_credits['cast'] = df_credits['cast'].apply(parse_dict_string)
df_cast = pd.json_normalize(df_credits['cast']).fillna('')

df_credits['crew'] = df_credits['crew'].apply(parse_dict_string)
df_crew = pd.json_normalize(df_credits['crew']).fillna('')

In [34]:
# Aplica la funcion "extraer_directores"
df_credits['directores'] = df_credits['crew'].apply(extraer_directores)

In [35]:
# Aplicar la función "unir_valores" a cada fila del DataFrame y crear nuevas columnas "id", "name" y "character", luego extrae estas columnas a un nuevo DF
df_cast[['id', 'name', 'character']] = df_cast.apply(lambda row: unir_valores(row, 'id', 'character'), axis=1, result_type='expand')
DF_cast = df_cast[['id', 'name', 'character']]

C:\Users\gasty\AppData\Local\Temp\ipykernel_13088\2444011566.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cast[['id', 'name', 'character']] = df_cast.apply(lambda row: unir_valores(row, 'id', 'character'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykernel_13088\2444011566.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_cast[['id', 'name', 'character']] = df_cast.apply(lambda row: unir_valores(row, 'id', 'character'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykerne

In [36]:
# Aplicar la función "unir_valores" a cada fila del DataFrame y crear nuevas columnas "id", "name" y "job", luego extrae estas columnas a un nuevo DF
df_crew[['id', 'name', 'job']] = df_crew.apply(lambda row: unir_valores(row, 'id', 'job'), axis=1, result_type='expand')
Df_Crew = df_crew[['id', 'name', 'job']]

C:\Users\gasty\AppData\Local\Temp\ipykernel_13088\3120580182.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_crew[['id', 'name', 'job']] = df_crew.apply(lambda row: unir_valores(row, 'id', 'job'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykernel_13088\3120580182.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_crew[['id', 'name', 'job']] = df_crew.apply(lambda row: unir_valores(row, 'id', 'job'), axis=1, result_type='expand')
C:\Users\gasty\AppData\Local\Temp\ipykernel_13088\3120580182.py:1:

In [37]:
# Elimina las columnas "cast" e "crew"
columnas_a_eliminar = ['cast', 'crew']
df_credits.drop(columnas_a_eliminar, axis=1, inplace=True)

# Modificacion de valores de las columnas y union de los df

In [38]:
# Cambia los valores de las columnas en el df original por los valores de las columnas especificada en los df auxiliares
df['belongs_to_collection'] = df_belongs_to_collection['name']
df['genres'] = DF_Genres['name']
df['production_companies'] = DF_Production_Companies['name']
df['production_countries'] = DF_Production_Countries['name']
df['spoken_languages'] = DF_Spoken_Languages['name']
df_credits['name_crew'] = df_crew['name']
df_credits['name_cast'] = df_cast['name']

In [40]:
# Resetea los indices de los df
df = df.reset_index(drop=True)
df_credits = df_credits.reset_index(drop=True)

In [44]:
# Cambia el formato de las columnas "id" a datos numericos
df['id'] = df['id'].astype(int)
df_credits['id'] = df_credits['id'].astype(int)

In [45]:
# Realizar la fusión de los DataFrames utilizando la columna "id" como clave
df_merged = pd.merge(df, df_credits, on='id')

In [46]:
# Df final
df_merged

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,status,tagline,title,vote_average,vote_count,return,release_year,directores,name_crew,name_cast
0,Toy Story Collection,30000000.0,"Animation,Comedy,Family",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,Pixar Animation Studios,United States of America,1995-10-30,...,Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,John Lasseter,"John Lasseter,Joss Whedon,Andrew Stanton,Joel ...","Tom Hanks,Tim Allen,Don Rickles,Jim Varney,Wal..."
1,,65000000.0,"Adventure,Fantasy,Family",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"TriStar Pictures,Teitler Film,Interscope Commu...",United States of America,1995-12-15,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,Joe Johnston,"Larry J. Franco,Jonathan Hensleigh,James Horne...","Robin Williams,Jonathan Hyde,Kirsten Dunst,Bra..."
2,Grumpy Old Men Collection,0.0,"Romance,Comedy",15602,en,A family wedding reignites the ancient feud be...,11.7129,"Warner Bros.,Lancaster Gate",United States of America,1995-12-22,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,1995,Howard Deutch,"Howard Deutch,Mark Steven Johnson,Mark Steven ...","Walter Matthau,Jack Lemmon,Ann-Margret,Sophia ..."
3,,16000000.0,"Comedy,Drama,Romance",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,Twentieth Century Fox Film Corporation,United States of America,1995-12-22,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,1995,Forest Whitaker,"Forest Whitaker,Ronald Bass,Ronald Bass,Ezra S...","Whitney Houston,Angela Bassett,Loretta Devine,..."
4,Father of the Bride Collection,0.0,Comedy,11862,en,Just when George Banks has recovered from his ...,8.387519,"Sandollar Productions,Touchstone Pictures",United States of America,1995-02-10,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,1995,Charles Shyer,"Alan Silvestri,Elliot Davis,Nancy Meyers,Nancy...","Steve Martin,Diane Keaton,Martin Short,Kimberl..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,NaN,0.0,NaN,30840,en,"Yet another version of the classic epic, with ...",5.683753,NaN,NaN,1991-05-13,...,Released,NaN,Robin Hood,5.7,26.0,0.000000,1991,John Irvin,"John Irvin,Sam Resnick,John McGrath,Sam Resnic...","Patrick Bergin,Uma Thurman,David Morrissey,Jür..."
45447,NaN,0.0,NaN,111109,tl,An artist struggles to finish his work while a...,0.178241,NaN,NaN,2011-11-17,...,Released,NaN,Century of Birthing,9.0,3.0,0.000000,2011,Lav Diaz,"Lav Diaz,Lav Diaz,Dante Perez,Lav Diaz,Lav Dia...","Angel Aquino,Perry Dizon,Hazel Orencio,Joel To..."
45448,NaN,0.0,NaN,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,NaN,NaN,2003-08-01,...,Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,2003,Mark L. Lester,"Mark L. Lester,C. Courtney Joyner,Jeffrey Gold...","Erika Eleniak,Adam Baldwin,Julie du Page,James..."
45449,NaN,0.0,NaN,227506,en,"In a small town live two brothers, one a minis...",0.003503,NaN,NaN,1917-10-21,...,Released,NaN,Satan Triumphant,0.0,0.0,0.000000,1917,Yakov Protazanov,"Yakov Protazanov,Joseph N. Ermolieff","Iwan Mosschuchin,Nathalie Lissenko,Pavel Pavlo..."


In [47]:
# Crea un archivo csv de los datos ya transformados
#df_merged.to_csv('datos.csv', index=False)